In [1]:
%gui qt
import os.path as op
import mne

sample_path = mne.datasets.sample.data_path()
data_path = op.join(sample_path, 'MEG', 'sample')
subjects_dir = op.join(sample_path, 'subjects')

In [2]:
inv = mne.minimum_norm.read_inverse_operator(op.join(data_path, 'sample_audvis-meg-oct-6-meg-fixed-inv.fif'))

Reading inverse operator decomposition from C:\Users\wmvan\mne_data\MNE-sample-data\MEG\sample\sample_audvis-meg-oct-6-meg-fixed-inv.fif...
    Reading inverse operator info...
    [done]
    Reading inverse operator decomposition...
    [done]
    305 x 305 full covariance (kind = 1) found.
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Noise covariance matrix read.
    7498 x 7498 diagonal covariance (kind = 2) found.
    Source covariance matrix read.
    Did not find the desired covariance matrix (kind = 6)
    7498 x 7498 diagonal covariance (kind = 5) found.
    Depth priors read.
    Did not find the desired covariance matrix (kind = 3)
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    P

In [3]:
raw = mne.io.read_raw_fif(op.join(data_path, 'sample_audvis_filt-0-40_raw.fif'))
events = mne.read_events(op.join(data_path, 'sample_audvis_filt-0-40_raw-eve.fif'))
event_id = {'audio/left': 1,
            'audio/right': 2,
            'visual/left': 3,
            'visual/right': 4}
epochs = mne.Epochs(raw, events, event_id)

Opening raw data file C:\Users\wmvan\mne_data\MNE-sample-data\MEG\sample\sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Current compensation grade : 0
288 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 4)
4 projection items activated


In [4]:
src = mne.read_source_spaces(op.join(subjects_dir, 'sample', 'bem', 'sample-all-src.fif'))

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read


In [5]:
epochs_stc = mne.minimum_norm.apply_inverse_epochs(epochs, inv, lambda2=0.1111)

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 3)
    Created the whitener using a noise covariance matrix with rank 302 (3 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Picked 305 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 288 (at most)
Processing epoch : 2 / 288 (at most)
Processing epoch : 3 / 288 (at most)
Processing epoch : 4 / 288 (at most)
Processing epoch : 5 / 288 (at most)
Processing epoch : 6 / 288 (at most)
Processing epoch : 7 / 288 (at most)
Processing epoch : 8 / 288 (at most)
Processing epoch : 9 / 288 (at most)
Processing epoch : 10 / 288 (at most)
Processing epoch : 11 / 288 (at most)
Processing epoch : 12 / 288 (at most)
Processing epoch : 13 / 288 (at most)
Processing epoch : 14 / 288 (at most)
Processing epoch : 15 / 28

Processing epoch : 203 / 288 (at most)
Processing epoch : 204 / 288 (at most)
Processing epoch : 205 / 288 (at most)
Processing epoch : 206 / 288 (at most)
Processing epoch : 207 / 288 (at most)
Processing epoch : 208 / 288 (at most)
Processing epoch : 209 / 288 (at most)
Processing epoch : 210 / 288 (at most)
Processing epoch : 211 / 288 (at most)
Processing epoch : 212 / 288 (at most)
Processing epoch : 213 / 288 (at most)
Processing epoch : 214 / 288 (at most)
Processing epoch : 215 / 288 (at most)
Processing epoch : 216 / 288 (at most)
Processing epoch : 217 / 288 (at most)
Processing epoch : 218 / 288 (at most)
Processing epoch : 219 / 288 (at most)
Processing epoch : 220 / 288 (at most)
Processing epoch : 221 / 288 (at most)
Processing epoch : 222 / 288 (at most)
Processing epoch : 223 / 288 (at most)
Processing epoch : 224 / 288 (at most)
Processing epoch : 225 / 288 (at most)
Processing epoch : 226 / 288 (at most)
Processing epoch : 227 / 288 (at most)
Processing epoch : 228 / 

In [6]:
import rsa
import numpy as np

In [16]:
model_dsm = np.array([
    [0, 0.5, 1, 1],
    [0.5, 0, 1, 1],
    [1, 1, 0, 0.5],
    [1, 1, 0.5, 0],
])
rsa_vals = rsa.rsa_source_level(epochs_stc, model_dsm, inv['src'], y=epochs.events[:, 2], n_folds=5, verbose=True)

100%|████████████████████████████████████████████| 577346/577346 [11:14:55<00:00, 14.26it/s]


ValueError: cannot reshape array of size 577346 into shape (7498,106,1)

In [14]:
rsa_vals.plot('sample', hemi='both', subjects_dir=subjects_dir)

Using control points [0.82807867 0.82807867 0.82807867]
<Brain subject_id="sample", hemi="both", surf="inflated">


In [9]:
inv.keys()

dict_keys(['methods', 'source_ori', 'nsource', 'nchan', 'coord_frame', 'units', 'source_nn', 'sing', 'eigen_leads_weighted', 'eigen_leads', 'eigen_fields', 'noise_cov', 'source_cov', 'orient_prior', 'depth_prior', 'fmri_prior', 'src', 'mri_head_t', 'info', 'nave', 'projs', 'proj', 'whitener', 'reginv', 'noisenorm'])

In [10]:
inv['src']

<SourceSpaces: [<surface (lh), n_vertices=155407, n_used=3732>, <surface (rh), n_vertices=156866, n_used=3766>] head coords, subject 'sample'>

In [11]:
epochs_stc[0]

<SourceEstimate  |  7498 vertices, subject : sample, tmin : -199.79521315838787 (ms), tmax : 499.48803289596964 (ms), tstep : 6.659840438612929 (ms), data shape : (7498, 106)>

In [12]:
len(inv['src'][0]['vertno']) + len(inv['src'][1]['vertno'])

7498

In [13]:
np.sum(inv['src'][0]['inuse']) + np.sum(inv['src'][1]['inuse'])

7498

In [ ]:
%debug

> c:\users\wmvan\projects\rsa\rsa\rsa.py(226)rsa_array()
    224             metric=rsa_metric,
    225             n_jobs=n_jobs)
--> 226         data = data.reshape((-1, X.shape[1], X.shape[2]) + data.shape[1:])
    227 
    228     elif spatial_radius is not None:

ipdb> data.shape
(577346, 1)
ipdb> X.shape
(288, 7498, 106)
ipdb> data.reshape(X.shape[1], X.shape[2], -1)
*** ValueError: cannot reshape array of size 577346 into shape (7498,106,newaxis)
ipdb> 7498 * 106
794788
ipdb> data.reshape(X.shape[1], -1, data.shape[1])
array([[[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[-0.20701967],
        [-0.41403934],
        [-0.41403934],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        ...,
        [-0.621059  ],
        [-0.621059  ],
        [-0.621059  ]],

       ..